In [213]:

from pyspark.sql import SparkSession

MAX_MEMORY = "12g"
spark = SparkSession.builder.appName("sql-query")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()
    
sc = spark.sparkContext

22/10/12 01:48:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [214]:
parquet_directory = "/Users/kuno/code/test/payhere/Q2/CAI_parquet1"
df1 = spark.read.parquet(f"{parquet_directory}")
parquet_directory = "/Users/kuno/code/test/payhere/Q2/CAI_parquet2"
df2 = spark.read.parquet(f"{parquet_directory}")

In [215]:
col_list = ["상가업소번호", "상호명", "지점명", "상권업종대분류코드", "상권업종대분류명", "상권업종중분류코드", "상권업종중분류명", "상권업종소분류코드", "상권업종소분류명", "표준산업분류코드", "표준산업분류명", "시도코드", "시도명", "시군구코드", "시군구명", "행정동코드", "행정동명", "법정동코드", "법정동명", "지번코드", "대지구분코드", "대지구분명", "지번본번지", "지번부번지", "지번주소", "도로명코드", "도로명", "건물본번지", "건물부번지", "건물관리번호", "건물명", "도로명주소", "구우편번호", "신우편번호", "동정보", "층정보", "호정보", "경도", "위도"]
i = 0
for col in col_list:
    i += 1
    df1 = df1.withColumnRenamed(col,f"col{i}") 
    df2 = df2.withColumnRenamed(col,f"col{i}") 


In [ ]:
df1.dtypes
df2.dtypes

In [217]:
df1.createOrReplaceTempView("CAI1")
df2.createOrReplaceTempView("CAI2")

In [218]:
query = "SELECT * FROM CAI2"
spark.sql(query).show(1)

+--------+--------------+----+----+---------+----+------------+------+----------+------+--------------+-----+------+-----+------+----------+-------+----------+------+-------------------+-----+-----+-----+-----+--------------------------+------------+--------------------+-----+-----+--------------------+-----+------------------------+------+-----+-----+-----+-----+----------------+----------------+
|    col1|          col2|col3|col4|     col5|col6|        col7|  col8|      col9| col10|         col11|col12| col13|col14| col15|     col16|  col17|     col18| col19|              col20|col21|col22|col23|col24|                     col25|       col26|               col27|col28|col29|               col30|col31|                   col32| col33|col34|col35|col36|col37|           col38|           col39|
+--------+--------------+----+----+---------+----+------------+------+----------+------+--------------+-----+------+-----+------+----------+-------+----------+------+-------------------+-----+-----+

In [219]:
# 시도 별 상가 점포수

query = """
SELECT col13 AS AREA,
       COUNT(col1) AS commercialcount
FROM CAI2
GROUP BY col13
"""

df = spark.sql(query)
df.show()

+--------------+---------------+
|          AREA|commercialcount|
+--------------+---------------+
|        경기도|         546552|
|    서울특별시|         346229|
|      충청남도|         118462|
|    부산광역시|         152901|
|      전라남도|         115271|
|      경상남도|         160655|
|      경상북도|         137554|
|      전라북도|         111568|
|    인천광역시|         115771|
|        강원도|         105784|
|      충청북도|          91856|
|    대구광역시|         104065|
|    광주광역시|          73821|
|    대전광역시|          73036|
|    울산광역시|          47907|
|세종특별자치시|          11932|
|제주특별자치도|          50630|
+--------------+---------------+



In [220]:
# 시군구 별 상가 점포수

query = """
SELECT col13 AS AREA,
       col15 AS AREA_m,
       COUNT(col1) AS commercialcount
FROM CAI2
GROUP BY col13, col15
"""

df = spark.sql(query)
df.show()

+----------+---------------+---------------+
|      AREA|         AREA_m|commercialcount|
+----------+---------------+---------------+
|    경기도|  용인시 처인구|          12555|
|서울특별시|         서초구|          20069|
|    경기도|       동두천시|           4762|
|    경기도|  안산시 상록구|          12614|
|서울특별시|         양천구|          11960|
|    경기도|         김포시|          20651|
|    경기도|  성남시 중원구|           9207|
|    경기도|고양시 일산동구|          12346|
|    경기도|         오산시|           9356|
|    경기도|         여주시|           6490|
|    경기도|         군포시|           7074|
|서울특별시|         관악구|          12606|
|서울특별시|         성북구|          11872|
|    경기도|         광명시|           9499|
|    경기도|         부천시|          29017|
|    경기도|         광주시|          17293|
|    경기도|         포천시|          10946|
|서울특별시|         금천구|           9715|
|    경기도|         안성시|           8893|
|    경기도|         과천시|           1997|
+----------+---------------+---------------+
only showing top 20 rows



In [221]:
# 전국 점포 유형

query = """
SELECT col5 AS commecial,
       COUNT(col1) AS commercialcount
FROM CAI2
GROUP BY col5
"""

df = spark.sql(query)
df.show()

+--------------+---------------+
|     commecial|commercialcount|
+--------------+---------------+
|          숙박|          41393|
|          음식|         797182|
|    생활서비스|         428890|
|          소매|         754830|
|        스포츠|          29281|
|관광/여가/오락|          53313|
|        부동산|          95870|
|     학문/교육|         163235|
+--------------+---------------+



In [222]:
# 전국 점포 세부 유형
query = """
SELECT col5 AS commecial,
       col7 AS commecial_m,
       COUNT(col1) AS commercialcount
FROM CAI2
GROUP BY col5, col7
"""

df = spark.sql(query)
df.show()

+--------------+-----------------------+---------------+
|     commecial|            commecial_m|commercialcount|
+--------------+-----------------------+---------------+
|          소매|        중고품소매/교환|           7697|
|    생활서비스|      개인/가정용품수리|          19815|
|          음식|            닭/오리요리|          49240|
|    생활서비스|              광고/인쇄|          15840|
|     학문/교육|           학문교육기타|           2946|
|          소매| 사진/광학/정밀기기소매|          31842|
|          소매|           종교용품판매|            410|
|관광/여가/오락|            스포츠/운동|            476|
|          소매|        의약/의료품소매|           9147|
|        스포츠|           실외운동시설|           2181|
|          소매|예술품/골동품/수석/분재|           3890|
|        스포츠|           실내운동시설|          24223|
|    생활서비스|                   사진|          15338|
|관광/여가/오락|     경마/경륜/성인오락|           1187|
|          음식|            커피점/카페|         110455|
|          소매|             기타판매업|          19707|
|     학문/교육|     학원-자격/국가고시|           2189|
|    생활서비스|        세탁/가사서

In [223]:
# 전국 점포 세부 유형

query = """
SELECT col5 AS commecial,
       col7 AS commecial_m,
       col9 AS commecial_s,
       COUNT(col1) AS commercialcount
FROM CAI2
GROUP BY col5, col7, col9
"""

df = spark.sql(query)
df.show()

+--------------+-----------------+------------------+---------------+
|     commecial|      commecial_m|       commecial_s|commercialcount|
+--------------+-----------------+------------------+---------------+
|    생활서비스|             사진|          상업사진|           1532|
|     학문/교육|학원-예능취미체육|  학원-펜글씨/한문|            289|
|     학문/교육|         학원기타|      운전연수전문|            165|
|관광/여가/오락|      스포츠/운동|    서바이벌게임장|             20|
|    생활서비스|     기타서비스업| 조감도/투시도전문|             11|
|          소매|    음/식료품소매|          식료품점|          22244|
|        스포츠|     운영관리시설|      기타운영설비|           2029|
|          소매|자동차/자동차용품|    중고타이어판매|            447|
|    생활서비스|         주택수리|          수도설비|           1473|
|    생활서비스|개인/가정용품수리|          악기수선|            120|
|          소매|운동/경기용품소매|인라인스케이트판매|             21|
|          음식|             한식|            냉면집|           3219|
|     학문/교육|         유아교육|   실내놀이터-유아|            713|
|          소매|         의복의류|   맞춤양장/양장점|           5839|
|          음식|     

In [224]:
# 시도별 점포 유형

query = """
SELECT col13 AS commercial,
       col5 AS AREA,
       COUNT(col1) AS count
FROM CAI2
GROUP BY col13, col5
"""

df = spark.sql(query)
df.show()

+----------+--------------+------+
|commercial|          AREA| count|
+----------+--------------+------+
|    경기도|          소매|170840|
|서울특별시|          음식|117033|
|    경기도|    생활서비스|101963|
|    경기도|        스포츠|  7690|
|서울특별시|        스포츠|  5175|
|서울특별시|        부동산| 17356|
|서울특별시|          소매|101805|
|    경기도|        부동산| 29132|
|    경기도|          음식|174428|
|서울특별시|관광/여가/오락|  8404|
|    경기도|     학문/교육| 43507|
|서울특별시|    생활서비스| 69233|
|서울특별시|     학문/교육| 24903|
|서울특별시|          숙박|  2320|
|    경기도|관광/여가/오락| 13148|
|    경기도|          숙박|  5844|
|  전라남도|          음식| 39428|
|  전라남도|          숙박|  3729|
|  전라남도|    생활서비스| 18335|
|  충청남도|          숙박|  3121|
+----------+--------------+------+
only showing top 20 rows



In [231]:
# 전국 업종, 분기별 점포 추이

query = """
SELECT t2.col7 AS commercial, t1.count AS Q1, t2.count AS Q2
FROM (SELECT col7, COUNT(col1) AS count FROM CAI2 GROUP BY col7 ) t2
JOIN (SELECT col7, COUNT(col1) AS count FROM CAI1 GROUP BY col7 ) t1
ON t2.col7 = t1.col7;
"""

df = spark.sql(query)
df.show()

+-----------------------+------+------+
|             commercial|    Q1|    Q2|
+-----------------------+------+------+
|          음/식료품소매|107572|111035|
|            커피점/카페|106862|110455|
|                   중식| 23731| 24304|
|     학원-자격/국가고시|  2206|  2189|
|            스포츠/운동|   393|   476|
|           기타서비스업| 11426| 17898|
|               유흥주점| 78822| 79477|
|      취미/오락관련소매|  4190|  4166|
|               학원기타| 33422| 35154|
|               가구소매| 14093| 14004|
|예술품/골동품/수석/분재|  3574|  3890|
|               의복의류| 92932| 95223|
|         놀이/여가/취미|  1288|  1458|
|                   분양|  2489|  8318|
|           이/미용/건강|164699|168230|
|                   사진| 13956| 15338|
|       선물/팬시/기념품| 25142| 27252|
|      개인/가정용품수리| 17906| 19815|
|     가정/주방/인테리어| 72541| 72728|
|            일식/수산물| 61427| 61355|
+-----------------------+------+------+
only showing top 20 rows



In [232]:
# 시도, 업종, 분기별 점포 추이

query = """
SELECT t2.col13 AS Area, t2.col7 AS commercial, t1.count AS Q1, t2.count AS Q2
FROM (SELECT col13, col7, COUNT(col1) AS count FROM CAI2 GROUP BY col13, col7 ) t2
JOIN (SELECT col13, col7, COUNT(col1) AS count FROM CAI1 GROUP BY col13, col7 ) t1
ON t2.col7 = t1.col7 AND t2.col13 = t2.col13;
"""

df = spark.sql(query)
df.show()

+--------------+-------------------+---+---+
|          Area|         commercial| Q1| Q2|
+--------------+-------------------+---+---+
|    대전광역시|           유아용품|194| 58|
|    광주광역시|           유아용품|194| 49|
|제주특별자치도|           유아용품|194| 27|
|세종특별자치시|           유아용품|194| 13|
|    울산광역시|           유아용품|194| 37|
|    인천광역시|           유아용품|194| 93|
|      전라북도|           유아용품|194| 90|
|      충청북도|           유아용품|194| 81|
|        강원도|           유아용품|194| 74|
|    대구광역시|           유아용품|194| 74|
|    부산광역시|           유아용품|194|116|
|      경상북도|           유아용품|194| 84|
|      경상남도|           유아용품|194| 88|
|      충청남도|           유아용품|194| 66|
|      전라남도|           유아용품|194| 66|
|        경기도|           유아용품|194|536|
|    서울특별시|           유아용품|194|190|
|세종특별자치시|페인트/유리제품소매|431| 11|
|    광주광역시|페인트/유리제품소매|431|150|
|제주특별자치도|페인트/유리제품소매|431| 96|
+--------------+-------------------+---+---+
only showing top 20 rows



In [306]:
# 광주광역시 업종별 개폐업 점포 수 및 추이

query = """
SELECT op.*, CLOSE, OPEN - CLOSE AS Transition 
FROM (SELECT t2.col7 AS commercial,
             COUNT(DISTINCT CASE WHEN t1.col1 IS NULL THEN t2.col1 ELSE NULL END) AS OPEN
      FROM (SELECT * FROM CAI2 WHERE col12 = 29) t2
      LEFT JOIN (SELECT * FROM CAI1 WHERE col12 = 29) t1
             ON t2.col1 = t1.col1 
      GROUP BY t2.col7) op
JOIN (SELECT t1.col7 AS commercial,
             COUNT(DISTINCT CASE WHEN t2.col1 IS NULL THEN t1.col1 ELSE NULL END) AS CLOSE
      FROM (SELECT * FROM CAI2 WHERE col12 = 29) t2
      RIGHT JOIN (SELECT * FROM CAI1 WHERE col12 = 29) t1
             ON t2.col1 = t1.col1 
      GROUP BY t1.col7) cl
ON op.commercial = cl.commercial
ORDER BY Transition DESC;
"""

df = spark.sql(query)
df.show()

+------------------+----+-----+----------+
|        commercial|OPEN|CLOSE|Transition|
+------------------+----+-----+----------+
|              한식| 406|  133|       273|
|          주택수리| 272|   32|       240|
|            대행업| 205|    1|       204|
|              분양| 164|    1|       163|
|      기타서비스업| 138|    8|       130|
|      이/미용/건강| 202|   80|       122|
|        부동산중개| 164|   48|       116|
|       커피점/카페| 184|   70|       114|
|     음/식료품소매| 178|   85|        93|
|          의복의류| 135|   54|        81|
|  선물/팬시/기념품|  87|   16|        71|
|        기타판매업|  62|    2|        60|
|          유흥주점| 117|   65|        52|
| 개인/가정용품수리|  54|    7|        47|
|   세탁/가사서비스|  62|   17|        45|
|        화장품소매|  62|   19|        43|
|          학원기타|  61|   19|        42|
| 자동차/자동차용품|  48|    9|        39|
|     자동차/이륜차|  55|   20|        35|
|인력/고용/용역알선|  36|    2|        34|
+------------------+----+-----+----------+
only showing top 20 rows

